In [69]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input,Session

In [35]:
bucket_name='banks-app'
my_region=boto3.session.Session().region_name
print(my_region)

eu-north-1


In [36]:
s3=boto3.resource('s3')
try:
    if my_region=='eu-north-1':
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
        #s3.create_bucket(Bucket=bucket_name)
    print("Bucket1 created sucessfully")
except Exception as e:
    print("s3 error:",e)

Bucket1 created sucessfully


In [38]:
prefix='xgboost-as-a-built-in-algo'
output_path='s3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)

s3://banks-app/xgboost-as-a-built-in-algo/output


In [43]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv","bank_clean.csv")
    print('Downloaded successfully')
except expection as e:
    print('Error is:',e)
try:
    model_data=pd.read_csv('./bank_clean.csv',index_col=0)
    print('Sucessfully loaded')
except exception as e:
    print('error while loading:',e)

Downloaded successfully
Sucessfully loaded


In [44]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [46]:
import numpy as np
train_data, test_data= np.split(model_data.sample(frac=1, random_state=24), [int(0.7*len(model_data))])
print(train_data.shape,test_data.shape)

(28831, 61) (12357, 61)


In [77]:
import os
pd.concat([train_data['y_yes'],train_data.drop(['y_yes','y_no'],axis=1)],axis=1).to_csv('train.csv',index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train=sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name,prefix),content_type='csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [78]:
pd.concat([test_data['y_yes'],test_data.drop(['y_yes','y_no'],axis=1)],axis=1).to_csv('test.csv',index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test=sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name,prefix),content_type='csv')


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [68]:
#building xgboost model - inbuilt
# models are stored in containers or image
#container=sagemaker.image_uris.retrieve(boto3.Session().region_name, 'xgboost', version='1.0-1')
from sagemaker import image_uris
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name,version ="latest")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [80]:
hyperparameters={"max-depth":"5", "eta":"0.2","gamma":"4","min_child_weight":"6","subsample":"0.7","objective":"binary:logistic","num_round":50}

In [81]:
estimator = sagemaker.estimator.Estimator(image_uri=container,
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_type = 'ml.m5.2xlarge',
                                          instance_count= 1,
                                          volume_size = 5,
                                          output_path=output_path,
                                           use_spot_instances=True,
                                           max_run=300,
                                           max_wait= 600)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [82]:

estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: xgboost-2024-01-02-10-38-12-883


2024-01-02 10:38:13 Starting - Starting the training job...
2024-01-02 10:38:32 Starting - Preparing the instances for training......
2024-01-02 10:39:40 Downloading - Downloading input data...
2024-01-02 10:40:10 Downloading - Downloading the training image...
2024-01-02 10:40:31 Training - Training image download completed. Training in progress...Arguments: train
[2024-01-02:10:40:49:INFO] Running standalone xgboost training.
[2024-01-02:10:40:49:INFO] File size need to be processed in the node: 4.83mb. Available memory size in the node: 23950.11mb
[2024-01-02:10:40:49:INFO] Determined delimiter of CSV input is ','
[10:40:49] S3DistributionType set as FullyReplicated
[10:40:49] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-01-02:10:40:49:INFO] Determined delimiter of CSV input is ','
[10:40:49] S3DistributionType set as FullyReplicated
[10:40:49] 12357x59 matrix with 729063 entries loaded from /opt/ml/input/data/

In [85]:
xgb_predictor=estimator.deploy(initial_instance_count=1,instance_type='ml.m5.2xlarge')

INFO:sagemaker:Creating model with name: xgboost-2024-01-02-11-07-18-788
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-01-02-11-07-18-788
INFO:sagemaker:Creating endpoint with name xgboost-2024-01-02-11-07-18-788


---!

In [88]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)
     

(12357,)


In [89]:
predictions_array

array([0.02635947, 0.10143997, 0.07695106, ..., 0.09779303, 0.0377324 ,
       0.03174843])

In [90]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))
     


Overall Classification Rate: 89.8%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10781)    37% (190)
Purchase        9% (1069)     63% (317) 



In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()